In [1]:
'''
generate the gts in pixelized cropping for simple based lline.
simple baseline aims at direct mapping from image space to camera coordinates.
A proper training requires even distribution of human subj in image plane.
It is not the case for many dataset, so we take the input from cropping plane.
save still in Nx n_jt x 3 
'''
from opt import opts, set_env
from os import path
import os
from tqdm import tqdm
import numpy as np
from utils.utils_pose import fliplr_joints, transform_joint_to_other_db
import json
import copy
from data.dataset import generate_patch_image, trans_point2d

candidate_sets = ['Human36M', 'ScanAva', 'MPII', 'MSCOCO', 'MuCo', 'MuPoTS', 'SURREAL']
for i in range(len(candidate_sets)):
	exec('from data.' + candidate_sets[i] + '.' + candidate_sets[i] + ' import ' + candidate_sets[i])

In [2]:
# make func
# def get_jt_hm_flt(ds, opts):

# read in ds,
dsNm = 'SURREAL'
phase = 'train'
dct_dsF = {'ScanAva':ScanAva, 'SURREAL':SURREAL} # func
dsF = dct_dsF[dsNm]       # ds func,, ScanAva or SURREAL
fd_sv = '/scratch/liu.shu/datasets/gts_3d_sbl'  # where to save th ds, for simple base line
if not path.exists(fd_sv):
	ds_fd = os.makedirs(fd_sv)

ds = dsF(phase, opts=opts) # keep a test set    all in  

Loading and augmenting ScanAva data...: 100%|██████████| 82000/82000 [00:09<00:00, 8456.15it/s] 


ScanAva train initialized


In [13]:
db = ds.data
N = len(db)
jts_hm =[]          # keep the 3d infor, 2d can be gooten from herer
jts_vis = []        # keep all visibilities

img_dm = np.zeros([256, 256,3]) # dummy image
# N=2 # for test
# we use patch , so 2d will be consistent with 3d hm, only regress depth
for idx in tqdm(range(N)):      # the gt one  
	data = copy.deepcopy(db[idx])
	bbox = data['bbox']  # give some default one for this in case not in there
	joint_img = data['joint_img']       # pelvis centered d 
	joint_vis = data['joint_vis']
	joint_cam = data['joint_cam']   # cam coordiante
	jt_img_ori = np.copy(joint_img) # keep a copy


	if False:   # for augmentation
		scale, rot, do_flip, color_scale, do_occlusion = get_aug_config()
	else:
		scale, rot, do_flip, color_scale, do_occlusion = 1.0, 0.0, False, [1.0, 1.0, 1.0], False
	# 3. crop patch from img and perform data augmentation (flip, rot, color scale, synthetic occlusion)
	img_patch, trans = generate_patch_image(img_dm, bbox, do_flip, scale, rot, do_occlusion,
	                                        input_shape=opts.input_shape)
	for i in range(len(joint_img)):  # 2d first for boneLen calculation
		joint_img[i, 0:2] = trans_point2d(joint_img[i, 0:2], trans)

	joint_img[:, 2] /= opts.bbox_3d_shape[0] / 2.   # fix box bone norm

	#
	for i in range(len(joint_img)):
		joint_img[i, 2] = (joint_img[i, 2] + 1.0) / 2.  # 0~1 normalize
		joint_vis[i] *= (
				(joint_img[i, 0] >= 0) & \
				(joint_img[i, 0] < opts.input_shape[1]) & \
				(joint_img[i, 1] >= 0) & \
				(joint_img[i, 1] < opts.input_shape[0]) & \
				(joint_img[i, 2] >= 0) & \
				(joint_img[i, 2] < 1)
		)  # nice filtering  all in range

	# change coordinates to output space the hmap space (usually 64 or 32 )
	joint_img[:, 0] = joint_img[:, 0] / opts.input_shape[1] * opts.output_shape[1]
	joint_img[:, 1] = joint_img[:, 1] / opts.input_shape[0] * opts.output_shape[0]
	joint_img[:, 2] = joint_img[:, 2] * opts.depth_dim  # 0 ~ depth_dim, 64 bin
# 	print('before trans', joint_img)
	if opts.ref_joints_name is not None:
		joint_img = transform_joint_to_other_db(joint_img, ds.joints_name, opts.ref_joints_name)
		joint_vis = transform_joint_to_other_db(joint_vis, ds.joints_name,
		                                        opts.ref_joints_name)  # invisible to no exist one
# 	print('after map', joint_img)
	joint_img = joint_img.astype(np.float32)  # x,y,z, hm
	joint_vis = (joint_vis > 0).astype(np.float32)
	# v_jt_2d = joint_img[:,2].flat()
	# v_jt_3d= joint_img.flat()

	# to result list
	jts_hm.append(joint_img)
	jts_vis.append(joint_vis)

	if False:        # for debug
		print('original jt img', jt_img_ori)
# 		print('bbox', bbox)
# 		print('trans', trans)
# 		print('new jt img', joint_img)

jts_hm = np.array(jts_hm)   # N x n_jts x 3, hm version 
jts_vis = np.array(jts_vis)
mu = jts_hm.mean(axis=0)    # each coordinate a mean
std = jts_hm.std(axis=0)
if True:
    print('mu' ,mu)
    print('std', std)

    

100%|██████████| 72000/72000 [02:04<00:00, 578.94it/s]


mu [[32.038452 33.551834 32.      ]
 [32.123062 33.46104  31.991486]
 [32.1789   44.99712  31.274212]
 [32.14009  58.500515 30.330101]
 [31.95418  33.642765 32.00851 ]
 [31.982546 45.324398 31.296698]
 [31.966042 58.718735 30.354013]
 [32.058365 24.039896 32.77383 ]
 [32.078304 14.527925 33.54777 ]
 [32.08232  11.724709 33.774406]
 [32.09057   6.117325 34.227875]
 [31.952795 16.409561 33.402596]
 [31.923128 21.881344 33.0328  ]
 [31.967    22.85951  33.06394 ]
 [32.192047 16.132776 33.369244]
 [32.28194  21.441196 32.9817  ]
 [32.31735  21.704811 33.05231 ]]
std [[ 6.266916   5.5956764  0.       ]
 [ 7.2981734  5.6444926  2.6398063]
 [ 6.7654014  5.427472   5.6569777]
 [ 9.218626   6.2607737  6.276914 ]
 [ 6.790136   5.5959053  2.639806 ]
 [ 6.9911556  5.3832116  6.2706633]
 [ 9.446077   6.3341594  6.85597  ]
 [ 4.026187   4.053258   3.1699257]
 [ 5.1323214  2.924651   6.339853 ]
 [ 5.8037596  2.5011854  7.1892285]
 [ 7.585558   2.0597336  8.956224 ]
 [ 6.3811097  3.4297724  6.8233905]

In [3]:
rst = {}
rst['jts_hm'] = jts_hm.tolist()
rst['jts_vis'] = jts_vis.tolist()
rst['mu'] = mu.tolist()
rst['std'] = std.tolist()
pth = path.join(fd_sv, '_'.join(dsNm,phase)+'.json')
with open(pth, 'w') as f:
	json.dump(rst, f)

100%|██████████| 72000/72000 [02:03<00:00, 584.93it/s]


mu [[32.192047 16.132776 33.369244]
 [32.1789   44.99712  31.274212]
 [32.123062 33.46104  31.991486]
 [32.038452 33.551834 32.      ]
 [32.14009  58.500515 30.330101]
 [31.95418  33.642765 32.00851 ]
 [31.982546 45.324398 31.296698]
 [32.28194  21.441196 32.9817  ]
 [31.923128 21.881344 33.0328  ]
 [32.31735  21.704811 33.05231 ]
 [31.967    22.85951  33.06394 ]
 [32.08232  11.724709 33.774406]
 [32.09057   6.117325 34.227875]
 [31.952795 16.409561 33.402596]
 [32.078304 14.527925 33.54777 ]
 [32.058365 24.039896 32.77383 ]
 [31.966042 58.718735 30.354013]]
std [[ 6.7134604  3.5882468  6.4710813]
 [ 6.7654014  5.427472   5.6569777]
 [ 7.2981734  5.6444926  2.6398063]
 [ 6.266916   5.5956764  0.       ]
 [ 9.218626   6.2607737  6.276914 ]
 [ 6.790136   5.5959053  2.639806 ]
 [ 6.9911556  5.3832116  6.2706633]
 [ 7.835144   5.0148516  8.297296 ]
 [ 7.6220984  4.8285303  8.779299 ]
 [ 9.553819  11.14119   10.90107  ]
 [ 9.270851  10.563285  11.215736 ]
 [ 5.8037596  2.5011854  7.1892285]

In [ ]:
# load in to check  
with open(pth,'r') as f:
	dt_in = json.load(f)
	
print(dt_in['mu'])


# loop adpat to all jts in cropping  in pixel
# flaten, column wise std and  mu.

